In [ ]:
from google.colab import drive
drive.mount('./mount')

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# MNIST DATA


### Mnist Dataset Class

In [ ]:
class MnistDataset(Dataset):

    def __init__(self, csv_file):
        self.data_df = pd.read_csv(csv_file, header=None)
        pass

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, index):
        # image target (label)
        label = self.data_df.iloc[index,0]
        target = torch.zeros((10))
        target[label] = 1.0

        # image data, normalised from 0-255 to 0-1
        image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values) / 255.0

        # return label, image data tensor and target tensor
        return label, image_values, target

    def plot_image(self, index):
        img = self.data_df.iloc[index,1:].values.reshape(28,28)
        plt.title("label = " + str(self.data_df.iloc[index,0]))
        plt.imshow(img, interpolation='none', cmap='Blues')
        pass

    pass

In [ ]:

#데이터 로드

mnist_dataset = MnistDataset('/content/mount/MyDrive/동훈/딥러닝 스터디/GAN/DATA/mnist_train.csv')

In [ ]:
mnist_dataset.plot_image(17)

In [ ]:
def generate_random(size):
    random_data = torch.rand(size)
    return random_data

### 판별자 네트워크

In [ ]:
class Discriminator(nn.Module):

    def __init__(self):
        # initialise parent pytorch class
        super().__init__()

        # define neural network layers
        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.Sigmoid(),
            nn.Linear(200, 1),
            nn.Sigmoid()
        )

        # create loss function
        self.loss_function = nn.MSELoss()

        # create optimiser, simple stochastic gradient descent
        self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01)

        # counter and accumulator for progress
        self.counter = 0;
        self.progress = []

        pass


    def forward(self, inputs):
        # simply run model
        return self.model(inputs)


    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)

        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 10000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, update weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass


    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5))
        pass

    pass

#### 판별기 테스트

In [ ]:
D = Discriminator()

for label, image_data_tensor, target_tensor in mnist_dataset:
    #실제 데이터
    D.train(image_data_tensor, torch.FloatTensor([1.0]))
    #생성된 데이터
    D.train(generate_random(784), torch.FloatTensor([0.0]))

In [ ]:
D.plot_progress()

In [ ]:
#임의로 선택한 이미지와 랜덤값(생성)을 학습한 모델에 넣어보면
for i in range(4):
  image_data_tensor = mnist_dataset[random.randint(0,60000)][1]
  print("학습 이미지 값 : ", D.forward( image_data_tensor ).item() )
  pass

for i in range(4):
  print("랜덤으로 생성한 값 : ", D.forward( generate_random(784) ).item() )
  pass

### 생성자 네트워크

In [ ]:
# generator class

class Generator(nn.Module):

    def __init__(self):
        # 부모 클래스 초기화
        super().__init__()

        # 모델 정의
        self.model = nn.Sequential(
            nn.Linear(1, 200),
            nn.Sigmoid(),
            nn.Linear(200, 784),
            nn.Sigmoid()
        )

        # 옵티마이저 설정
        self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01)

        # 로깅 변수 설정 및 초기화
        self.counter = 0;
        self.progress = []

        pass


    def forward(self, inputs):
        return self.model(inputs)


    def train(self, D, inputs, targets):
        # 이미지 생성
        g_output = self.forward(inputs)

        # 생성 이미지를 판별자에 통과
        d_output = D.forward(g_output)

        # Loss 계산
        loss = D.loss_function(d_output, targets)

        # 로깅
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass

        # 업데이트
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass


    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5))
        pass

    pass


#### 생성자 테스트

In [ ]:
G = Generator()

import numpy as np
#랜덤값 설정
output = G.forward(generate_random(1))

#784차원 reshape
img = output.detach().numpy().reshape(28,28)

#시각화
plt.imshow(img, interpolation='none', cmap = 'Blues')


### GAN 학습

In [ ]:
%%time
#네트워크 재정의
D = Discriminator()
G = Generator()

#학습
for label, image_data_tensor, target_tensor in mnist_dataset:

    #정상 이미지 -> 1로 판별
    D.train(image_data_tensor, torch.FloatTensor([1.0]))

    #가짜 이미지 -> 0로 판별
    D.train(G.forward(generate_random(1)).detach(), torch.FloatTensor([0.0]))

    #생성자 학습 - 생성한 이미지를 진짜 처럼 학습
    G.train(D, generate_random(1), torch.FloatTensor([1.0]))

    pass

In [ ]:
D.plot_progress()

그래프 해석
- 손실은 0에 가까워지다가 갑자기 0.25로 치솟는 부분은 판별자와 생성자가 균형이 맞기 시작했다는 것을 의미
- 이후에는 판별자가 좀 더 앞서나가게 되고 0에 가까울수록 생성자의 성능이 떨어져서 더는 판별자를 속일 수 없다는 상태

In [ ]:
G.plot_progress()

그래프 해석
- 초기 손실값이 치솟는 점은 판별자가 생성자에서 나온 이미지를 잘 구별하는 것
- 이후 손실이 0.25로 하락하는 것은 균형이 잘 맞는 상태

### 이미지 생성(추론)

In [ ]:
f, axarr = plt.subplots(2,3, figsize=(16,8))
for i in range(2):
    for j in range(3):
        output = G.forward(generate_random(1))
        img = output.detach().numpy().reshape(28,28)
        axarr[i,j].imshow(img, interpolation='none', cmap='Blues')
        pass
    pass

### 모드 붕괴
* 위와 같이 어떠한 값이 들어와도 같은 모양의 이미지를 생성하게 되는 것을 의미함
* 생성자가 판별자보다 앞서갔을 경우 가장 잘 만들었던 이미지를 계속해서 만들어 내는 것이라고 생각해볼 수 있음
* 이러한 문제를 해결하기 위해서는 판별자를 생성자보다 더 많이 훈련시킬수 있지만 효과는 없다
* 결국 훈련의 질이 중요하다